## Scrapy - imdb film verisi


Bu sayfadaki kodları hazırlarken takip eden kaynaktan faydalandım: https://app.datacamp.com/learn/courses/web-scraping-with-python

Daha önce yapmadıysanız önce ```pip install``` komutunu kullanarak kütüphaneyi indirin.

In [ ]:
pip install scrapy

In [ ]:
# Scrapy kütüphanesinden Selector metodunu çağırın
from scrapy import Selector

# requests kütüphanesini indirin
import requests

url = 'https://m.imdb.com/chart/top'

# HTML kaynağını içeren html string'ini oluşturun
html = requests.get(url).content

# html objesini kullanarak bir Selector objesi tanımlayın
sel = Selector(text = html)

# HTML belgesindeki öğelerin sayısını yazdırın
print( "You have found: ", len(sel.xpath('//*')))

In [ ]:
type(html)

In [ ]:
type(sel)

In [ ]:
xpath_for_movienames ='//*[@id="chart-content"]/div/div/div/a/span/h4'

Şimdi tanımladığımız xpath'i ve yukarda tanımladığımız Selector objesini kullanarak programın ilgili konuma gidip xpath ile tarif edilen bölümleri getirmesini sağlayın. 

In [ ]:
sel.xpath(xpath_for_movienames).extract()

In [ ]:
len(sel.xpath(xpath_for_movienames).extract())

In [ ]:
# VEYA

xpath_for_movienames ='//h4/text()'

Bu listeyi bir değişken olarak tanımlayabilirsiniz.

In [ ]:
movies = sel.xpath(xpath_for_movienames).extract()

In [ ]:
movies

#### "Strip" fonksiyonu ile elimizdeki metni temizleyelim

Bunu yapmadığımızda bir çok boş ve gereksiz karekteri saklamış ve taşımış olacağız. ```Strip``` fonksiyonunu kullanarak bir döngü yaratabilir, ve bu işlemi hızlıca gerçekleştirebilirsiniz.

In [ ]:
movie_list=[]
for string in movies:
    cleaned_string = string.strip()
    if cleaned_string != '':
        movie_list.append(cleaned_string)
    
print(movie_list)

İndirdiğiniz ve sakladığınız listenin uzunluğuna göz atın. IMDB listesine göre en beğenilen 250 filmi indirmiş olmalısınız. Doğru sayıya ulaştığınızdan emin olun.

In [ ]:
len(movie_list)

#### Filmlerin sıralamasını ve yıllarını çekelim

Önce filmlerin sıralamasını gösteren xpath'i tanımlamanız, ardından Selector objesini kullanarak bu bölümleri getirmeniz gerekiyor.

In [ ]:
xpath_for_movieorder ='//h4/span[1]/text()'
movie_order = sel.xpath(xpath_for_movieorder).extract()

In [ ]:
xpath_for_moviedates ='//h4/span[2]/text()'
movie_date = sel.xpath(xpath_for_moviedates).extract()

#### Filmlerin linkleri

>
>

__İlk film için Xpath'ler:__

- //*[@id="chart-content"]/div[1]/div[1]/div/a

- //*[@id="chart-content"]/div[1]/div[2]/div/a

- //*[@id="chart-content"]/div[2]/div[1]/div/a

- //*[@id="chart-content"]/div[2]/div[2]/div/a

…


In [ ]:
xpath_for_movielink = '//*[@id="chart-content"]/div/div/div/a/@href'
movie_link = sel.xpath(xpath_for_movielink).extract()

In [ ]:
len(movie_link)

In [ ]:
movie_link

İndirdiğiniz link parçası açmak istediğiniz linkin sadece bir bölümünü içeriyor. O halde bu bölümü linkin kayıp olan kısmıyla birleştirmeniz gerekiyor. Bunu bir döngü yaratarak yapabilirsiniz.

The Shawshank Redemption için film linki: 


https://m.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=df09bbba-7a44-41c0-bc85-426ba05a5574&pf_rd_r=1TZHFKVGSKZZKVA8TW56&pf_rd_s=top-1&pf_rd_t=15506&pf_rd_i=top&ref_=m_chttp_tt_1

In [ ]:
first_part_url = 'https://m.imdb.com'
last_part_url = '?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=df09bbba-7a44-41c0-bc85-426ba05a5574&pf_rd_r=R9N3Q0473JZET8YH4S2A&pf_rd_s=top-1&pf_rd_t=15506&pf_rd_i=top&ref_=m_chttp_tt_1'

In [ ]:
movie_link_merged = []

for i in movie_link:
    link = first_part_url + i + last_part_url
    movie_link_merged.append(link)

In [ ]:
movie_link[5]

In [ ]:
movie_link_merged[5]

In [ ]:
len(movie_link_merged)

#### İndirdiklerimizi saklayalım

Bunun için ilk olarak bir sözlük yaratabilirsiniz.

In [ ]:
dictimdb = {'movie_order':movie_order, 'movie_date':movie_date, 'movie_list':movie_list, 'movie_link_merged': movie_link_merged}

Fakat pandas dataframe'i sözlükten daha efektif bir şekilde kullanılabilir ve görüntülenebilir. Şimdi elinizdeki sözlüğü bir dataframe'e çevirin.

In [ ]:
import pandas as pd

In [ ]:
data_imdb = pd.DataFrame(dictimdb)

In [ ]:
data_imdb

In [ ]:
data_imdb.to_csv("IMDB_Filmlerim.csv")
data_imdb.to_json("IMDB_Filmlerim.json")

#### Peki oyuncular

Biraz daha ileri gidip bu filmler için oyuncuları da indirmek istersek ne yapmalıyız? En başta tanımladığımız URL'yi hatırlayın. Bunun için sayfadaki ilk hücreye bakmanız yeterli. Elinizdeki 250 tane URL'yi teker teker açılış yapacağınız URL olarak tanımlarsanız, her seferinden baştan yeni bir Selector objesi tanımlayarak ilgili sayfaları dolaşabilir ve aradığınız oyuncu listesini depolayabiliriz.

Döngü oluşturmadan önce sistemin düzgün çalıştğını kontrol etmek amaçlı ilk film için oyuncu listesini çekmeyi deneyin.

In [ ]:
data_imdb['movie_link_merged'][0]

In [ ]:
url = data_imdb['movie_link_merged'][0]

In [ ]:
html = requests.get(url).content

sel = Selector(text = html)

print( "You have found: ", len(sel.xpath('//*')))

>
>
__İlk dört oyuncu için kopyaladığımız Xpath'ler:__

- //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[1]/div[2]/a

- //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[2]/div[2]/a

- //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[3]/div[2]/a

- //*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[4]/div[2]/a


In [ ]:
xpath_for_movieplayers ='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div/div[2]/a/text()'
movie_players_first_movie = sel.xpath(xpath_for_movieplayers).extract()

In [ ]:
movie_players_first_movie

Şimdi bunu 250 URL'yi tarayarak yapın. Bulduğunuz sonuçları ```players_list``` isimli bir listede saklayabilirsiniz.

In [ ]:
players_list=[]

for i in range(250):
    url = data_imdb['movie_link_merged'][i]
    html = requests.get(url).content
    sel = Selector(text = html)
    movie_players = sel.xpath(xpath_for_movieplayers).extract()
    players_list.append(movie_players)

Bir üst hücredeki komut her bir link için döngüyü en baştan başlatıyor, bu da ciddi bir zaman kaybı demek. Bunun için şimdilik sadece ilk 5 filme bakabiliriz.

Çoklu sayfalarda bu işlemi yaparken zamandan tasarruf etmek için spider'ı kullanmalısınız.

In [ ]:
players_list=[]

for i in range(3):
    url = data_imdb['movie_link_merged'][i]
    html = requests.get(url).content
    sel = Selector(text = html)
    movie_players = sel.xpath(xpath_for_movieplayers).extract()
    players_list.append(movie_players)

In [ ]:
players_list